# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Adtividad Semana 5

### **Vectores Embebidos Pre-entrenados: Fasttext**

#### **Nombres y matrículas de los integrantes del equipo:**



*   Renzo Zagni A01795457
*   Angel Iván Ahumada Arguelles A00398508
*   Karina Zafra Vallejo A01793979
*   Ernesto Zapata Garza A00915255



In [2]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:

import os
os.environ['USE_CYTHON'] = '0'
!pip install Cython
!pip install fasttext

import re
import numpy as np
import pandas as pd

import nltk
import spacy
import fasttext
import fasttext.util

from nltk.corpus import stopwords, words , wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score


In [5]:
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
fasttext.util.download_model('en', if_exists='ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


'cc.en.300.bin'

##**Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [11]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


from google.colab import drive
drive.mount('/content/drive')
DIR = "/content/drive/MyDrive/Colab Notebooks/NLP3"
os.chdir(DIR)

dfa = pd.read_csv('amazon5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi = pd.read_csv('imdb5.txt', sep='\s{2,}(?=\d)', names=['review','label'], header=None, engine ='python', encoding='utf-8')
dfy = pd.read_csv('yelp5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')

df = pd.concat([dfa, dfi, dfy], ignore_index=True)

# *********** Aquí termina la sección de agregar código *************


Mounted at /content/drive


In [12]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3000 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [13]:
# Y veamos sus primeros registros:

df.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


##**Pregunta - 2:**

Realiza el proceso de limpieza.

Aplica el preprocesamiento que consideres adecuado, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización.











In [14]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Separamos la información:
X = df.review     # Serie de strings
Y = df.label      # Serie de enteros 0s y 1s

# Como se vio en el ejercicio pasado, tenemos un problema con los renglones 1125
# y 1788 donde el comentario no utiliza caracteres alfabeticos (10/10). Si no
# hacemos algo al respecto,  cuando solo consideremos caracteres alfabeticos,
# estos renglones se convertiran en elementos vacios los que a  su vez
# se asociaran a vectores nulos . Esto nos acarreara problemas al momento  de
# procesarlos.
# Como estamos hablando de ocurrencias muy bajas (2 de 3000) procederemos a
# eliminar renglones vacios luego del procesamiento

# Consideremos la siguiente lista de palabras asociada a negaciones en inglés:
negwords = [ 'no', 'nor', 'not', 'ain', 'aren', "aren't", 'don', "don't", 'couldn',
             "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't",
             'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn',
             "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't",
             'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won',
             "won't", 'wouldn', "wouldn't"]
mystopwords =  [x for x in  stopwords.words('english') if x not in negwords]

nltk_words = set(words.words())
nlp = spacy.load("en_core_web_sm")

def clean_tok(doc):
  # Caracteres Alfabeticos
  # Incluimos los caracteres del español asi como los caracteres ê y å los cuales
  # se encuentran en los comentarios y considero importante preservarlos en este
  # momento.
  wds = re.sub(r'[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑêÊÅå]',r' ' , doc)
  #
  # Eliminamos palabras con longitud mayor a 1
  wds = re.sub(r'\b\w\b', '', wds)
  #
  # Elimamos espacios en blanco duplicados
  wds =  re.sub(r'\s+', ' ', wds).strip()
  #
  # Aplicamos tecnica de lemmatizacion utilizando libreria spacy
  tokens = [wd.lemma_ for wd in nlp(wds)]
  #
  # Tokenizamos los strings y convertimos los token a minusculas
  tokens = [ token.lower() for token in tokens]
  #
  # Eliminamos Stopwords
  tokens = [token for token in tokens if token not in mystopwords]
  #
  # Eliminamos caracteres duplicados no validos
  tokens = [clean_dup_chars(x) for x in tokens]

  return tokens

#
# Metodo para eliminar caracteres duplicados. Es una funcion agresiva y en
# algunos casos no podra eliminar correctamente los caracteres duplicados invalidos
# dependiendo de que tan erronea es la palabra
#
def clean_dup_chars(word):

 if word not in nltk_words:                                         # No es una palabra valida en ingles
   if re.search(r'(.)\1', word):                                    # Tiene caracteres duplicados
     word = re.sub(r'([a-z])\1', r'\1', word[::-1], count=1)[::-1]  # Eliminamos un character duplicado
     return clean_dup_chars(word)                                   # Repetimos el proceso
   else:
    return word
 else:
  return word

# Aplicamos los metodos de limpieza
#
Xclean = [clean_tok(x) for x in X ]

# Eliminamos los renglones vacios resultado del proceso de limpieza
for i, k in enumerate(Xclean):
  if len(k) == 0:
    del Xclean[i]
    del Y[i]

# *********** Aquí termina la sección de agregar código *************

In [15]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x in Xclean[0:5]:
  print(x)


['no', 'way', 'plug', 'us', 'unless', 'go', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'jawbone']
['tie', 'charger', 'conversation', 'last', 'minute', 'major', 'problems']
['mic', 'great']


#**Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [16]:

# ************* Inicia la sección de agregar código:*****************************

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)


# *********** Termina la sección de agregar código *************

# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2098 2098
X,y Val: 450 450
X,y Test 450 450


#**Pregunta - 4:**



Construye tu vocabulario a continuación


In [17]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

midiccionario = Counter()

for k in range(len(x_train)):
  midiccionario.update(x_train[k])

# *********** Aquí termina la sección de agregar código *************

In [18]:
# b.	Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario generado:')


# ******* Inicia la sección de agregar código: ***********


len(midiccionario)


# *********** Aquí termina la sección de agregar código *************

Longitud del vocabulario generado:


3332

c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

El diccionario debe generase únicamente con los datos de entrenamientopara evitar el filtrado de información (data leakage) ya que si se utiliza el conjunto de validación y/o de prueba para construir el vocabulario, el modelo se podría estar indirectamente entrenando en estos datos, comprometiendo su imparcialidad.

### ++++++++ Termina la sección de agregar texto: +++++++++++


In [19]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Filtramos cada conjunto con las palabras de midiccionario

# Como el diccionario for creado a partir del conjunto de entrenamiento, no es
# necesario ejecutar el filtrado. Creamos una copia profunda en caso de que
# debamos modificar una de las listas sin afectar la otra
train_x = x_train[:]

val_x = []
for ss in x_val:
  val_x.append([w for w in ss if w in midiccionario])

test_x = []
for ss in x_test:
  test_x.append([w for w in ss if w in midiccionario])

#
# Validamos que no tengamos renglones vacios en validacion y prueba

for i, k in enumerate(val_x):
  if len(k) == 0:
    del val_x[i]
    y_val.drop(y_val.index[i],inplace=True)

for i, k in enumerate(test_x):
  if len(k) == 0:
    del test_x[i]
    y_test.drop(y_test.index[i],inplace=True)

print('X,y Train:', len(train_x), len(y_train))
print('X,y Val:', len(val_x), len(y_val))
print('X,y Test', len(test_x), len(y_test))

# *********** Aquí termina la sección de agregar código *************


X,y Train: 2098 2098
X,y Val: 450 450
X,y Test 447 447


In [20]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

['pay', 'bill', 'not', 'tip', 'feel', 'server', 'terrible', 'job']
['predictable', 'not', 'bad', 'watch']
['love', 'great', 'armband']
['watch', 'film', 'want', 'learn', 'work', 'artist']
['translate', 'movie', 'living', 'first', 'movie', 'year', 'working', 'experience', 'find', 'offensive', 'intelligence']


#**Pregunta - 5:**


a. Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de Google y Glove de Stanford.

### ++++++++ Inicia la sección de agregar texto: +++++++++++




| **Modelo**    | **Pros**                                                                 | **Contras**                                                             |
|---------------|--------------------------------------------------------------------------|------------------------------------------------------------------------|
| **FastText**  | - Mejor representación de palabras raras.                                | - Mayor consumo de memoria debido a la inclusión de subpalabras.        |
|               | - Mejor manejo de errores ortográficos y variaciones en la escritura.    | - Más lento en entrenamiento y predicción comparado con Word2Vec.       |
|               | - Representación de palabras mejorada debido a la información de subpalabras.| - Incremento en la complejidad y tamaño del modelo debido a los n-grams.                     |
|               |                                                                          |                                                                        |
| **Word2Vec**  | - Entrenamiento rápido y eficiente, adecuado para datasets largos.       | - No maneja bien palabras raras o fuera del vocabulario.                |
|               | - Captura las relaciones semánticas efectivas.                           | - No considera la estructura interna de las palabras (subpalabras).     |
|               | - Menor consumo de memoria comparado con FastText.                       | - Ignora el orden de las palaras .                    |
|               | - Uso de modelos de DeepLearning con CBOW y Skip-gram.                   |   |
| **GloVe**     | - Captura mejor las estadísticas globales del corpus.                    | - Requiere gran cantidad de memoria para almacenar la matriz de coocurrencia. |
|               | - Basado en estadísticas globales de co-ocurrencia, lo que mejora las representaciones. | - Entrenamiento más lento debido al cálculo de la matriz de coocurrencia. |
|               | - Resultados consistentes y robustos en varias tareas NLP.               | - Menos flexible en términos de adaptación a datos nuevos y menos efectivo si se tiene un corpora pequeño. |
|               | - Representaciones más estables comparado con Word2Vec.                  | - Difícil de actualizar con nuevo vocabulario sin reentrenar desde cero. |

### ++++++++ Termina la sección de agregar texto: +++++++++++


#**Pregunta - 6:**

Utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300 para generar un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el “valor” será su vector embebido de dimensión 300.

Este diccionario deberá ser del mismo tamaño que el vocabulario previo que hayas construido previamente.

Es recomendable que una vez que generes el nuevo vocabulario de vectores embebidos, guardes dicho diccionario en un archivo.

Recuerda borrar la variable donde descargaste los 2 millones de vectores embebidos Fasttext.



In [21]:
# # ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

diccionario_npz = 'nlp_semana05.npz'

def save_dicc(dicc, filename):
    keys = np.array(list(dicc.keys()))
    values = np.array(list(dicc.values()))
    np.savez(filename, keys=keys, values=values)

def load_dicc(filename):
    data = np.load(filename)
    keys = data['keys']
    values = data['values']
    dictionary = dict(zip(keys, values))
    return dictionary

ft = fasttext.load_model('cc.en.300.bin')
ft_dim = ft.get_dimension()
midicc = {}

for k , w in enumerate(midiccionario):
  midicc.update({w:ft.get_word_vector(w)})

save_dicc(midicc, 'nlp_semana05.npz')
del(ft)

print("Longitud del diccionario                       : " + str(len(midiccionario)))
print("Longitud del diccionario con vectores embebidos: " + str(len(midicc)))
# *********** Aquí termina la sección de agregar código *************

Longitud del diccionario                       : 3332
Longitud del diccionario con vectores embebidos: 3332


In [22]:
!ls -l

total 18537467
-rw------- 1 root root      58226 May 16 21:38 amazon5.txt
-rw------- 1 root root 7237176312 May 19 23:42 cc.en.300.bin
-rw------- 1 root root 4503593528 May 19 23:42 cc.en.300.bin.gz
-rw------- 1 root root          2 May 20 17:27 FastText.joblib
-rw------- 1 root root      86022 May 16 21:39 imdb5.txt
-rw------- 1 root root    4212154 May 23 03:50 nlp_semana05.npz
-rw------- 1 root root 7237176312 May 20 00:18 this.file
-rw------- 1 root root      61320 May 16 21:39 yelp5.txt


#**Pregunta - 7:**



Generamos los vectores embebidos a paertir de los conjuntos de entrenamiento, validación y preuba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [23]:
# # ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Detokenizamos cada renglones
train_x_docs = []
for k in range(len(train_x)):
  train_x_docs.append(' '.join(train_x[k]))

val_x_docs = []
for k in range(len(val_x)):
  val_x_docs.append(' '.join(val_x[k]))

test_x_docs = []
for k in range(len(test_x)):
  test_x_docs.append(' '.join(test_x[k]))

#
# Generamos las lista de promedio de vectores de los comentarios
train_avg_vectors = []
for cnt, comment in enumerate(train_x):
  comment_vectors = []
  for word in comment:
    comment_vectors.append(midicc[word])
  train_avg_vectors.append([np.mean(x) for x in zip(*comment_vectors)])

val_avg_vectors = []
for cnt, comment in enumerate(val_x):
  comment_vectors = []
  for word in comment:
    comment_vectors.append(midicc[word])
  val_avg_vectors.append([np.mean(x) for x in zip(*comment_vectors)])

test_avg_vectors = []
for cnt, comment in enumerate(test_x):
  comment_vectors = []
  for word in comment:
    comment_vectors.append(midicc[word])
  test_avg_vectors.append([np.mean(x) for x in zip(*comment_vectors)])

# Generamos los data frames con los vectores embebidos
trainEmb = pd.DataFrame(columns=train_x_docs, data=np.array(train_avg_vectors).T)
valEmb = pd.DataFrame(columns=val_x_docs, data=np.array(val_avg_vectors).T)
testEmb = pd.DataFrame(columns=test_x_docs, data=np.array(test_avg_vectors).T)

In [24]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

Train-Emb: (300, 2098)
Val-Emb: (300, 450)
Test-Emb: (300, 447)


#**Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [25]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

modeloLR = LogisticRegression(C = 0.1,
                              random_state = 1)

modeloLR.fit(trainEmb.values.T, y_train)

print('Logistic Regression: Train-accuracy: %.2f%%' % (100*modeloLR.score(trainEmb.values.T, y_train)))
print('Logistic Regression: Val-accuracy: %2.f%%' % (100*modeloLR.score(valEmb.values.T, y_val)))

# *********** Aquí termina la sección de agregar código *************


Logistic Regression: Train-accuracy: 82.13%
Logistic Regression: Val-accuracy: 82%


In [26]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

modeloRF = RandomForestClassifier(n_estimators = 150,
                                  min_samples_split = 10,
                                  min_samples_leaf = 4,
                                  max_leaf_nodes = 20,
                                  bootstrap = True,
                                  n_jobs = -1,
                                  random_state = 1)

modeloRF.fit(trainEmb.values.T,y_train)

print('Random Forest: Train-accuracy: %.2f%%' % (100*modeloRF.score(trainEmb.values.T, y_train)))
print('Random Forest: Val-accuracy: %2.f%%' % (100*modeloRF.score(valEmb.values.T, y_val)))


# *********** Aquí termina la sección de agregar código *************

Random Forest: Train-accuracy: 89.66%
Random Forest: Val-accuracy: 82%


#**Pregunta - 9:**



Reporte del mejor modelo.


In [27]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Ambos modelos tienen la misma presicion con el conjunto de validacion pero el modelo de Randon Forest
# quedo un poco sobre entrenado por los que decidimos seleccionar el modelo de Regression Logaritmica
mejor_modelo = modeloLR

print('Test-accuracy con el mejor modelo %.2f%%' % (100*mejor_modelo.score(testEmb.values.T, y_test)))

from sklearn.metrics import confusion_matrix

pred = mejor_modelo.predict(testEmb.values.T)
print('\nMatriz de confusión con el mejor modelo:')
print(confusion_matrix(y_test, pred, labels=[0,1]))

print('\nMatriz de confusión con el mejor modelo en proporciones:')
print(confusion_matrix(y_test, pred, labels=[0,1]) / (pred.shape[0] / 100))

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
print()
print("Accuracy : " + str(int(accuracy_score(y_test,pred) * 100)) + "%")
print("Precision: " + str(int(precision_score(y_test,pred) * 100 ))+ "%")
print("Recall   : " + str(int(recall_score(y_test,pred)* 100))+ "%")
print("F1       : " + str(int(f1_score(y_test,pred)*100))+ "%")


# *********** Aquí termina la sección de agregar código *************

Test-accuracy con el mejor modelo 82.33%

Matriz de confusión con el mejor modelo:
[[183  35]
 [ 44 185]]

Matriz de confusión con el mejor modelo en proporciones:
[[40.93959732  7.82997763]
 [ 9.84340045 41.38702461]]

Accuracy : 82%
Precision: 84%
Recall   : 80%
F1       : 82%


#**Pregunta - 10:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

En esta asignación, se implementaron vectores embebidos utilizando el modelo FastText para la representación de texto y se entrenaron dos modelos: uno de regresión logística y otro de bosques aleatorios.

Los resultados obtenidos fueron los siguientes:

- Modelo de Regresión Logística:
  - Precisión con datos de entrenamiento: 82.13%
  - Precisión con datos de validación: 82%

- Modelo de Bosques Aleatorios:**
  - Precisión con datos de entrenamiento: 89.66%
  - Precisión con datos de validación: 82%

El uso de FastText para crear vectores embebidos tiene ventajas importantes sobre los modeles utilizados la semana pasada --conteo de palabras (TF) y TF-IDF--. El modelo FastText contiene vectores pre-entrenados de palabras que facilitan el procesamiento de relaciones y contexto entre las palabras.

A diferencia de los métodos basados en conteo o TF-IDF, que solo cuentan la frecuencia de las palabras sin considerar su contexto, FastText tiene en cuenta el contexto de las palabras y utiliza vectores que reflejan mejor las similitudes entre ellas.


Al comparar los dos modelos entrenados, la regresión logística demostró ser un modelo robusto con una precisión mas consistente con datos de entrenamiento así como con los datos de validación. Esto es un indicador de que el modelo no quedo sobre-entrenado y que generalizo la data de validación de forma consistente. Por otro lado, aunque el modelos de bosques aleatorios mostró una mayor precisión en los datos de entrenamiento (89%), esta alta precisión no se tradujo en los datos de validación, donde la precisión fue similar a la del modelo de regresión logística (82%). Esto sugiere que los bosques aleatorios se sobre-entreno, capturando o aprendiendo de los patrones específicos del conjunto de entrenamiento que no se generalizan bien a nuevos datos.


### ++++++++ Termina la sección de agregar texto: +++++++++++

##**Fin de la Actividad de vectores Embebidos - FastText**